In [ ]:
import random

from deap import base
from deap import creator
from deap import tools

import pandas as pd

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
# cambiar list por una clase custom
# una clase custom que tenga una secuencia aleatorio de POS dadas en data.csv
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def evaluate(individual):
  # return a tuple
  return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
# mu = mean value, sigma = standart deviation, indpb = Independent probability for each attribute to be mutated
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
# tournsize = The number of individuals participating in each tournament
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [ ]:
population = toolbox.population(n=50)
mate_probability = 0.5
mutation_probability = 0.2
number_of_generations = 40
# Evaluate the entire population
fitnesses = map(toolbox.evaluate, population)

df = pd.DataFrame(population)
df

In [ ]:
for individual, fitness in zip(population, fitnesses):
  individual.fitness.values = fitness

In [ ]:
for generation in range(number_of_generations):
  # Select the next generation individuals
  offspring = toolbox.select(population, len(population))

  # Clone the selected individuals
  offspring = map(toolbox.clone, offspring)
  offspring = list(offspring
  )
  # Apply crossover and mutation on the offspring
  for child1, child2 in zip(offspring[::2], offspring[1::2]):
    if random.random() < mate_probability:
      toolbox.mate(child1, child2)
      del child1.fitness.values
      del child2.fitness.values

  for mutant in offspring:
    if random.random() < mutation_probability:
      toolbox.mutate(mutant)
      del mutant.fitness.values

  # Evaluate the individuals with an invalid fitness
  invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
  fitnesses = map(toolbox.evaluate, invalid_ind)
  for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit

  # The population is entirely replaced by the offspring
  
  population[:] = offspring

In [ ]:
df = pd.DataFrame(population)
df